In [ ]:
import operator
import math
import pandas as pd
import sys
import ctypes
import tkinter as tk
from random import randint
import warnings
from datetime import datetime, timedelta
from functools import reduce
from PIL import ImageChops
import pyautogui
import numpy as np
import cv2
import time
from PIL import ImageGrab
import pyautogui as pag
from PIL import Image
import requests
TOKEN = '5330229825:AAEegysQ-gssCV-Jgdf6bferx9SGyn7mC8C1TdfY' #токен бота телеграм (несуществующий)
URL = 'https://api.telegram.org/bot'


def send_msg(text, chat_id=99999): # chat_id ваш ID телеграм для получения уведомлений
    try:
        chat_id = "814715"
        url_req = URL + TOKEN + "/sendMessage" + \
            "?chat_id=" + chat_id + "&text=" + text
        results = requests.get(url_req)
    except:
        print('ошибка отправки уведомления')


def send_photo_file(img, chat_id=99999):   # chat_id ваш ID телеграм для получения уведомлений
    try:
        files = {'photo': open(img, 'rb')}
        requests.post(f'{URL}{TOKEN}/sendPhoto?chat_id={chat_id}', files=files)
    except Exception as e:
        print(e)


def find_koord(file): # функция возвращает координаты фрагмента переданного в аргументе
    full_screen = pyautogui.screenshot()
    full_screen.save("screen.png")
    img = cv2.imread("screen.png")  # картинка, на которой ищем объект
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # преобразуем её в серую
    # объект, который преобразуем в серый, и ищем его на gray_img
    template = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    w, h = template.shape[::-1]  # инвертируем из (y,x) в (x,y)

    result = cv2.matchTemplate(gray_img, template, cv2.TM_CCOEFF_NORMED)
    loc = np.where(result == np.max(result))
    # рисует прямоугольник вокруг объекта
    for pt in zip(*loc[::-1]):
        cv2.rectangle(img, pt, (pt[0] + w, pt[1] + h), (0, 255, 0), 3)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    koord = (max_loc[0]+(w/2), max_loc[1]+(h/2))
    return koord

# функция возвращает координаты фрагмента переданного в аргументе, а так же точность совпадения
# точность нахождения можно использовать для оценки найден ли фрагмент на скриншоте или нет
def find_koord_2(file):   
    full_screen = pyautogui.screenshot()
    full_screen.save("screen.png")
    img = cv2.imread("screen.png")  # картинка, на которой ищем объект
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # преобразуем её в серую
    # объект, который преобразуем в серый, и ищем его на gray_img
    template = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    w, h = template.shape[::-1]  # инвертируем из (y,x) в (x,y)

    result = cv2.matchTemplate(gray_img, template, cv2.TM_CCOEFF_NORMED)
    loc = np.where(result == np.max(result))
    # рисует прямоугольник вокруг объекта
    for pt in zip(*loc[::-1]):
        cv2.rectangle(img, pt, (pt[0] + w, pt[1] + h), (0, 255, 0), 3)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    koord = (max_loc[0]+(w/2), max_loc[1]+(h/2))
    return koord, np.max(result)

if platform == "darwin":
    platform_kor = 0.5
else:
    platform_kor = 1


def find_click(img):
    k = find_koord(img)
    pag.click(x=k[0]*platform_kor, y=k[1]*platform_kor)  # умножение на 0.5 для mac os. 


def click(x, y): # клик по найденым ранее координатам
    pag.click(int(x*platform_kor), int(y*platform_kor))  # умножение на 0.5 для mac os. 

    
# ниже пример простого цикла. При обноружении определенных фрагментов осуществляет клик по ним.
for i in range(12):
    res = 0
    res2 = 0
    while res < 0.9 and res2 < 0.9:
        time.sleep(10)
        k, res = find_koord_2('tap1.PNG')
        k2, res2 = find_koord_2('tap3.PNG')
    if res>0.9:
        time.sleep(6)    
        click(x=k[0], y=k[1])
        time.sleep(3)
        find_click('tap2.PNG')
    if res2>0.9:
        time.sleep(6)    
        click(x=k2[0], y=k2[1])
        time.sleep(3)
        find_click('tap4.PNG')
    send_msg('Клик от сна res = ' + str(res)+ ' res2 = '+str(res2))
    send_photo_file('screen.PNG')
    time.sleep(300)

